# Strategy Learner getting data online


Kairoart 2018
"""


## Overview

In this project we will design a learning trading agent. Your choices are:

1. Regression or classification-based learner: Create a strategy using your Random Forest learner. Suggestions if you follow this approach: Classification_Trader_Hints. Important note, if you choose this method, you must set the leaf_size for your learner to 5 or greater. This is to avoid degenerate overfitting in-sample.
2. Reinforcement Learner-based approach: Create a Q-learning-based strategy using your Q-Learner. Read the Classification_Trader_Hints first, because many of the ideas there are relevant for the Q trader, then see Q_Trader_Hints
3. Optimization-based learner: Create a scan-based strategy using an optimizer. Read the Classification_Trader_Hints first, because many of the ideas there are relevant for the Opto trader, then see Opto_Trader_Hints

Regardless of your choice above, your learner should work in the following way:

* In the training phase (e.g., addEvidence()) your learner will be provided with a stock symbol and a time period. It should use this data to learn a strategy. For instance, for a regression-based learner it will use this data to make predictions about future price changes.
* In the testing phase (e.g., testPolicy()) your learner will be provided a symbol and a date range. All learning should be turned OFF during this phase.


If the date range is the same as used for the training, it is an in-sample test. Otherwise it is an out-of-sample test. Your learner should return a trades dataframe like it did in the last project. Here are some important requirements: Your testPolicy() method should be much faster than your addEvidence() method. The timeout requirements (see rubric) will be set accordingly. Multiple calls to your testPolicy() method should return exactly the same result. 

## Tasks

* Devise numerical/technical indicators to evaluate the state of a stock on each day.
* Build a strategy learner based on one of the learners described above that uses the indicators.
* Test/debug the strategy learner on specific symbol/time period problems.
* Write a report describing your learning strategy.

## Data Details, Dates and Rules

* For your report, trade only the symbol JPM. This will enable us to more easily compare results. We will test your learner with other symbols as well.
* You may use data from other symbols (such as SPY) to inform your strategy.
* The in sample/development period is January 1, 2008 to December 31 2009.
* The out of sample/testing period is January 1, 2010 to December 31 2011.
* Starting cash is 100,000.
* Allowable positions are: 1000 shares long, 1000 shares short, 0 shares.
* Benchmark: The performance of a portfolio starting with 100,000 cash, investing in 1000 shares of the symbol in use and holding that position. Include transaction costs.
* There is no limit on leverage.
* Transaction costs: Commission will always be 0.00, Impact may vary, and will be passed in as a parameter to the learner.
* Minimize use of herrings.

## Implement Strategy Learner

For this part of the project you should develop a learner that can learn a trading policy using your learner. You should be able to use your Q-Learner or RTLearner from the earlier project directly, with no changes. If you want to use the optimization approach, you will need to create new code or that. You will need to write code in StrategyLearner.py to "wrap" your learner appropriately to frame the trading problem for it.

### StrategyLearner API

import StrategyLearner as sl
learner = sl.StrategyLearner(verbose = False, impact = 0.000) # constructor
learner.addEvidence(symbol = "AAPL", sd=dt.datetime(2008,1,1), ed=dt.datetime(2009,12,31), sv = 100000) # training phase
df_trades = learner.testPolicy(symbol = "AAPL", sd=dt.datetime(2010,1,1), ed=dt.datetime(2011,12,31), sv = 100000) # testing phase

The input parameters are:

* verbose: if False do not generate any output
* impact: The market impact of each transaction.
* symbol: the stock symbol to train on
* sd: A datetime object that represents the start date
* ed: A datetime object that represents the end date
* sv: Start value of the portfolio

The output result is:

* df_trades: A data frame whose values represent trades for each day. Legal values are +1000.0 indicating a BUY of 1000 shares, -1000.0 indicating a SELL of 1000 shares, and 0.0 indicating NOTHING. Values of +2000 and -2000 for trades are also legal when switching from long to short or short to long so long as net holdings are constrained to -1000, 0, and 1000.

## Goal

Implement a StrategyLearner that trains a QLearner for trading a symbol.

## Import libraries

In [6]:
import pandas as pd
import numpy as np  
import datetime as dt

# To fetch data
from pandas_datareader import data as pdr   
import fix_yahoo_finance as yf  
yf.pdr_override()   

from util import create_df_benchmark, get_data, fetchOnlineData, get_data_av, normalize_data
from strategyLearner import strategyLearner
from marketsim import compute_portvals_single_symbol, market_simulator
from indicators import get_momentum, get_sma, get_sma_indicator, compute_bollinger_value, get_RSI, plot_cum_return,  plot_momentum, plot_sma_indicator, plot_rsi_indicator, plot_momentum_sma_indicator, plot_stock_prices, plot_norm_data_vertical_lines

# TA Library (https://github.com/bukosabino/ta)
from ta import *

# Add plotly for interactive charts
import plotly
from plotly.offline import iplot
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools


## Initial Variables

In [7]:
start_val = 100000
symbol = "AABA"
commission = 0.00
impact = 0.0
num_shares = 1000

## In-sample performance

Show the performances of portfolio and benchmark in the in-sample period.

In [8]:
# Specify the start and end dates for this period.

start_d = dt.datetime(2018, 4, 20)
#end_d = dt.datetime(2018, 10, 30)
yesterday = dt.date.today() - dt.timedelta(1)
end_d = dt.datetime(2019, 4, 19) 

### Get portfolio data from Yahoo

In [9]:
#portf_value = fetchOnlineData(start_d, end_d, symbol)
#portf_value = fetchOnlineData(start_d, symbol, dt.date.today())

## Get Alpha Vantage data

In [10]:
# Import data from Alpha Vantage
dates = pd.date_range(start_d, dt.date.today())
portf_value = get_data_av(symbol, dates, del_cols=False)

## TA Indicators

In [11]:
# Create datafrane with all TA indicators
df = add_all_ta_features(portf_value, "Open", "High", "Low", "Close", "Volume", fillna=True)
df.info()

/home/emi/PycharmProjects/factors/lib/python3.7/site-packages/ta/trend.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/emi/PycharmProjects/factors/lib/python3.7/site-packages/ta/trend.py:174: RuntimeWarning:

invalid value encountered in double_scalars

/home/emi/PycharmProjects/factors/lib/python3.7/site-packages/ta/trend.py:634: FutureWarning:

Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning

/home/emi/PycharmProjects/factors/lib/python3.7/site-packages/ta/trend.py:656: FutureWarning:

Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning



<class 'pandas.core.frame.DataFrame'>
Index: 251 entries, 2018-04-20 to 2019-04-18
Data columns (total 65 columns):
Open                       251 non-null float64
High                       251 non-null float64
Low                        251 non-null float64
Close                      251 non-null float64
Adj Close                  251 non-null float64
Volume                     251 non-null float64
volume_adi                 251 non-null float64
volume_obv                 251 non-null float64
volume_obvm                251 non-null float64
volume_cmf                 251 non-null float64
volume_fi                  251 non-null float64
volume_em                  251 non-null float64
volume_vpt                 251 non-null float64
volume_nvi                 251 non-null float64
volatility_atr             251 non-null float64
volatility_bbh             251 non-null float64
volatility_bbl             251 non-null float64
volatility_bbm             251 non-null float64
volatility_bbhi     

In [12]:
normalize_data(df)

,Open,High,Low,Close,Adj Close,Volume,volume_adi,volume_obv,volume_obvm,volume_cmf,...,momentum_mfi,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,others_dr,others_dlr,others_cr
date,,,,,,,,,,,,,,,,,,,,,
2018-04-20,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,...,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN
2018-04-23,0.993992,0.986831,0.987109,0.983535,0.983535,3.134816,-inf,-inf,NaN,NaN,...,1.000000,-inf,1.000000,1.000000,1.000000,1.000000,NaN,-inf,-inf,-inf
2018-04-24,0.985748,0.980732,0.969826,0.974669,0.974669,0.519761,-inf,-inf,NaN,NaN,...,1.000000,-inf,1.000000,1.000000,1.000000,1.000000,NaN,-inf,-inf,-inf
2018-04-25,0.961995,0.954394,0.950418,0.960737,0.960737,0.683142,inf,-inf,NaN,NaN,...,1.000000,-inf,1.000000,1.000000,1.000000,1.000000,NaN,-inf,-inf,-inf
2018-04-26,0.962694,0.968732,0.975069,0.982269,0.982269,0.717755,inf,inf,NaN,NaN,...,1.000000,-inf,1.000000,1.000000,1.000000,1.000000,NaN,inf,inf,-inf
2018-04-27,0.990499,0.985445,0.984842,0.981283,0.981283,1.740018,-inf,-inf,NaN,NaN,...,1.000000,-inf,1.000000,1.000000,1.000000,1.000000,NaN,-inf,-inf,-inf
2018-04-30,0.980858,0.981425,0.983709,0.986209,0.986209,0.957893,-inf,inf,NaN,NaN,...,1.000000,-inf,1.000000,1.000000,1.000000,1.000000,NaN,inf,inf,-inf
2018-05-01,0.970379,0.974494,0.982292,0.983535,0.983535,1.405336,inf,-inf,NaN,NaN,...,1.000000,-inf,1.000000,1.000000,1.000000,1.000000,NaN,-inf,-inf,-inf
2018-05-02,0.979740,0.989604,0.992067,0.988038,0.988038,1.072884,-inf,inf,NaN,NaN,...,1.000000,-inf,1.000000,1.000000,1.000000,1.000000,NaN,inf,inf,-inf


## Plot stock values

### Stock prices

Stock prices from Jan 2008 to yesterday.

In [10]:
from markupsafe import Markup
plot_prices = plot_stock_prices(df['Adj Close'], symbol, output_type='nb')
iplot(plot_prices)

PlotlyError: The `figure_or_data` positional argument must be `dict`-like, `list`-like, or an instance of plotly.graph_objs.Figure

### Momentum

In [7]:
# Normalize the prices Dataframe
normed = pd.DataFrame()
normed['Adj Close'] = portf_value['Adj Close'].values / portf_value['Adj Close'].iloc[0];

In [8]:
# Compute momentum
sym_mom = get_momentum(normed['Adj Close'], window=10)


In [9]:
# 3. Plot raw AMZN values and Momentum
plot_momentum = plot_momentum(portf_value.index, normed['Adj Close'], sym_mom, "Momentum Indicator", (12, 8), output_type='nb')

ValueError: 
    Invalid value of type 'pandas.core.series.Series' received for the 'name' property of scatter
        Received value: 0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10     0.050664
11     0.080299
12     0.083137
13     0.072437
14     0.087004
15     0.079498
16     0.016620
17     0.023209
18     0.047953
19     0.099127
20     0.086459
21     0.052154
22     0.027385
23     0.009744
24     0.034999
25     0.054208
26     0.090611
27     0.101242
28     0.060028
29     0.019261
         ...   
226    0.028195
227    0.028536
228    0.001880
229   -0.000104
230    0.030247
231    0.054865
232    0.039844
233    0.028882
234    0.004020
235   -0.006595
236    0.023037
237    0.022427
238    0.030297
239    0.028290
240    0.024304
241    0.027108
242    0.041018
243    0.076887
244    0.118891
245    0.088826
246    0.062037
247    0.066141
248    0.044292
2

In [11]:
start_d = dt.datetime(2017, 12, 11)
end_d = dt.datetime(2018, 9, 26)


# Get benchmark data
benchmark_prices = fetchOnlineData(start_d, end_d, symbol)

# Create benchmark data: Benchmark is a portfolio starting with $100,000, investing in 1000 shares of symbol and holding that position
df_benchmark_trades = create_df_benchmark(symbol, start_d, end_d, num_shares)

#print (df_benchmark_trades)

# Train and test a StrategyLearner
# Set verbose to True will print out and plot the cumulative return for each training epoch
stl = strategyLearner(num_shares=num_shares, impact=impact, 
                      commission=commission, verbose=True,
                      num_states=3000, num_actions=3)
stl.add_evidence(symbol=symbol, start_val=start_val, 
                 start_date=start_d, end_date=end_d)
df_trades = stl.test_policy(symbol=symbol, start_date=start_d,
                            end_date=end_d)
print (df_trades)

TypeError: create_df_benchmark() takes 2 positional arguments but 4 were given

In [ ]:
# Retrieve performance stats via a market simulator
orders_count, sharpe_ratio, cum_ret, std_daily_ret, avg_daily_ret, final_value, cum_ret_bm, avg_daily_ret_bm, std_daily_ret_bm, sharpe_ratio_bm, final_value_bm, portvals, portvals_bm, df_orders  = market_simulator(df_trades, df_benchmark_trades, symbol=symbol, start_val=start_val, commission=commission, impact=impact)

plot_norm_data = plot_norm_data_vertical_lines(
                        df_orders,
                        portvals,
                        portvals_bm,
                        vert_lines=False,
                        title="Training Portfolio Value",
                        xtitle="Dates",
                        ytitle="Value ($)")

### Training market simulator


In [ ]:
# Retrieve performance stats via a market simulator
print ("Performances during training period for {}".format(symbol))
print ("Date Range: {} to {}".format(start_d, end_d))
orders_count, sharpe_ratio, cum_ret, std_daily_ret, avg_daily_ret, final_value = market_simulator(df_trades, df_benchmark_trades, symbol=symbol, 
                 start_val=start_val, commission=commission, impact=impact, title="Portfolio Value", xtitle="Dates", ytitle="Value")

## Out of sample performance

Show the performances of portfolio and benchmark in the out of sample period. Use the same StrategyLearner trained above and retrieve a trades dataframe via test_policy.

In [ ]:
# Specify the start and end dates for this period.
start_d = dt.datetime(2010, 1, 1)
end_d = dt.datetime(2011, 12, 31)

# Get benchmark data
benchmark_prices = get_data([symbol], pd.date_range(start_d, end_d), addSPY=False).dropna()

# Create benchmark data: Benchmark is a portfolio starting with $100,000, investing in 1000 shares of symbol and holding that position
df_benchmark_trades = create_df_benchmark(symbol, start_d, end_d, num_shares)

#print (df_benchmark_trades)

# Test a StrategyLearner
# Use the same StrategyLearner trained above and retrieve a trades dataframe via test_policy
df_trades = stl.test_policy(symbol=symbol, start_date=start_d,
                            end_date=end_d)
#print (df_trades)



### Test market simulator


In [ ]:
# Retrieve performance stats via a market simulator
print ("Performances during test period for {}".format(symbol))
print ("Date Range: {} to {}".format(start_d, end_d))
orders_count, sharpe_ratio, cum_ret, std_daily_ret, avg_daily_ret, final_value = market_simulator(df_trades, df_benchmark_trades, symbol=symbol, 
                 start_val=start_val, commission=commission, impact=impact, title="Portfolio Value", xtitle="Dates", ytitle="Value")

In [ ]:
df_trades.info()

# Report

Describe the steps you took to frame the trading problem as a learning problem for your learner. What are your indicators? Did you adjust the data in any way (dicretization, standardization)? Why or why not?

**Steps:**

1. Get adjusted close prices for symbol.
2. Set the QLearner parameters:
    * num_shares: The number of shares that can be traded in one order
    * epochs: The number of times to train the QLearner
    * num_steps: The number of steps used in getting thresholds for the
    discretization process. It is the number of groups to put data into.
3. Compute technical indicators and use them as features to be fed into a Q-learner.
    * Momentum
    * SMA indicator
    * RSI indicator
4. Get features and thresholds.
5. Define states, actions, and rewards. 
    * States are combinations of our features. 
    * Actions are buy, sell, do nothing. 
    * Rewards:
        Calculate the daily reward as a percentage change in prices: 
        - Position is long: if the price goes up (curr_price > prev_price),
          we get a positive reward; otherwise, we get a negative reward
        - Position is short: if the price goes down, we get a positive reward;
        otherwise, we a negative reward
        - Position is cash: we get no reward
6. Set initial position holding to nothing.
7. Create a series that captures order signals based on actions taken.
8. Iterate over the data by date.
    * Discretize features and return a state. Get a state; add 1 to position so that states >= 0.
    * On the first day, get an action without updating the Q-table.
    * On the last day, close any open positions.
    * Add new_pos to orders.
    * Update current position.
9. Create a trade dataframe.
10. Training: Choose the training period and you iterate over that training period and update your Q-table on each iteration. When you reach the end of that training period you backtest to see how good the model is and you go back and repeat, until the model quits getting better.
Once it's converged you stop, you've got your model.
11. Testing the model: You just backtest it on later data.

    

## Indicator charts

### Momentum chart

In [ ]:
# Specify the start and end dates for this period.
start_d = dt.datetime(2008, 1, 1)
end_d = dt.datetime(2009, 12, 31)

# Set dates
dates = pd.date_range(start_d, end_d)

# Get adjusted close prices for symbol
df = get_data([symbol], dates, addSPY=False)
df = df.dropna()


# Normalize the prices Dataframe
normed = pd.DataFrame()
for column in df:
    normed[column] = df[column].values / df[column].iloc[0];

# 2. Compute momentum
sym_mom = get_momentum(normed[column], window=10)

# 3. Plot raw JPM values and Momentum
plot_momentum(df.index, normed[column], sym_mom, "Momentum Indicator", (12, 8))

### SMA indicator

In [ ]:
# Specify the start and end dates for this period.
start_d = dt.datetime(2008, 1, 1)
end_d = dt.datetime(2009, 12, 31)

# Set dates
dates = pd.date_range(start_d, end_d)

# Get adjusted close prices for symbol
df = get_data([symbol], dates, addSPY=False)
df = df.dropna()

# Normalize the prices Dataframe
normed = pd.DataFrame()
for column in df:
    normed[column] = df[column].values / df[column].iloc[0];

# Compute SMA
sma_JPM, q = get_sma(normed[column], window=10)

# Plot symbol values, SMA and SMA quality
plot_sma_indicator(dates, df.index, normed[column], sma_JPM, q, "Simple Moving Average (SMA)")


### Momentum/SMA cross indicator

In [ ]:
# Specify the start and end dates for this period.
start_d = dt.datetime(2008, 1, 1)
end_d = dt.datetime(2009, 12, 31)

# Set dates
dates = pd.date_range(start_d, end_d)

# Get adjusted close prices for symbol
df = get_data([symbol], dates, addSPY=False)
df = df.dropna()

# Compute momentum
sym_mom = get_momentum(normed[column], window=10)

# Compute SMA
sma_JPM, q = get_sma(normed[column], window=10)

# Plot symbol values, SMA and Momentum
plot_momentum_sma_indicator(dates, df.index, normed[column], sma_JPM, sym_mom, "Momentum/SMA")



### RSI indicator

In [ ]:
# Specify the start and end dates for this period.
start_d = dt.datetime(2008, 1, 1)
end_d = dt.datetime(2009, 12, 31)

# Set dates
dates = pd.date_range(start_d, end_d)

# Get adjusted close prices for symbol
df = get_data([symbol], dates, addSPY=False)
df = df.dropna()

# 1. Compute RSI
rsi_JPM = get_RSI(df[symbol]).values.astype(float)
#print(rsi_JPM.)
# 2. Plot RSI
plot_rsi_indicator(dates, df.index, df[symbol], rsi_JPM, window=14, 
                   title="RSI Indicator", fig_size=(12, 6))


# TODO

Write a PDF report describing your system. The centerpiece of your report should be the description of how you utilized your learner to determine trades